# ROOT软件操作模式介绍
本内容纯为个人爱好总结，相关内容以root官网为准[https://root.cern/doc/master/index.html](https://root.cern/doc/master/index.html)

1. 终端模式（进入root软件的terminal通过键入有关命令简单的查看、检验数据）
2. 对话框模式（进入root软件后，键入TBrowser b）
3. 脚本模式（通过编写脚本文件，实现数据的批量分析、处理、画图、存图等功能）

**注意**
- 脚本模式指令与终端模式命令基本相似，但脚本模式某写功能比终端模式更严格，需要作些准备工作！
  - 比如终端模式tree->Draw()即可实现画图；脚本模式测需要创建画布、填图、画图；
- 终端模式与对话框模式下，对tree->Draw()的操作可通过鼠标实现；
- 脚本模式下，对tree->Draw()的操作仅能键入对应的命令才能实现；

## 终端模式常用命令
**(对于tree->Draw()的具体操作单独介绍)**

- 打开root文件

```c++
    root -l path+name.root
```

- 查看当前root中结构信息：root文件名字、tree结构名字、存储histgram-1D/2D/3D等信息	

```c++
	.ls
```

- 查看tree结构信息，即tree中Branch具体内容,为后面画图作准备
  - 此处tree为.ls执行之后，TTree之后的tree的名字，此处的tree就叫tree；
  
```c++
    tree->Print();
```

- 读取tree中第event number个事件的信息；	

```c++
    tree->Show([event number])
```

- 浏览tree中第event number个事件的信息；

```c++
    tree->Scan("Branchname1:Branchname2:...:Branchnamen","","",numbers of events, start events);
```
- 画tree结构中Branch的histogram
  - 如tree->Draw("branch_y:branch_x>>alias(xbin,minx,maxx,ybin,miny,maxy)","condition,cut etc.","colz etc.");  
  - 第一个双引号：内容+座标轴范围与bin，alias为该条件别名；
  - 第二个双引号：画图的条件、符合、CUT等；
  - 第三个双引号：option，如colz等；
  
```c++
	tree->Draw("ctof:td-tu>>(2000,-20,50,1000,0,100)","PID==0","colz");
	tree->Draw("td-tu>>htx(500,-20,50)","","");
```

- 画root中以画布形式存储（非tree结构）histogram图；

```c++
	hctof->Draw();//hctof可从.ls命令查看
```
- excute outer command in terminal of root
  - ![shell command]
  - eg:!ls -lh treeADC.*

- 退出root软件
  - .q

## 对话框模式
**(进入后有关拟合、cut、坐标轴、曲线等功能可通过鼠标实现)**

- 进入对话框模式

```c++
	TBrowser b
```

## 脚本模式
**利用C++语言，借由ROOT库函数，自己编写代码（.c/.cpp/.h等等）用于数据分析处理以及批量画图、存图等；**
- 编写分析代码并命名：如filename.c;
- 在终端键入如下命令：

```c++
root -l filename.c
```



# 脚本模式相关功能代码介绍

## 创建带有tree结构的root文件大致流程
1. 主函数体**(文件名要与主函数名称相同)**；
2. 常量声明，一般包括物理常量/探测器常量；
3. tree结构中Branch所需的变量声明；
4. 定义新的root文件，声明新的tree；
5. 将变量地址添加到tree中，其tree中Branch与变量地址关联；还可以定义hisrogram；
6. 循环体，计算各种变量，并填入数据；
7. 将数据写入histogram/tree，存储并关系root文件；

详细代码见李老师数据分析课homework1.1中的[tree.cc](https://dragon-xi.github.io/zhli-data-ana-homework/homework1.1/tree.cc)

**代码示例**

```c++
////主函数声明
    void/Int_t main(){}

////变量声明
    Int_t; Double_t; Long64_t 等等；

////创建名为tree.root的文件，指针为*opf,与opf->Close()匹配使用，告知程序.root文件处理完成/关闭；
	TFile * opf = new TFile("tree.root","recreate");
////创建新tree，名称为tree，其指针为*opt;
	TTree * opt = new TTree("tree","tree structure");

////为opt指针创建Branch("变量名称",变量指针,"变量名/变量数据格式")
////将之前声明的变量指针与Branch链接一起，注意此处Branch相关变量要与之前声明的变量对应；
	opt->Branch("x",&x,"x/D");
////按照opt->Branch(" ",	," ")(一般为多个Branch)所定义的参数向opt指针填入全部数据；
	opt->Fill();//该命令一般放入循环体中
////创建histogram,TH1D、TH2D、TH3D、TH1F、TH2F、TH3F等
	TH1D * hctof = new TH1D("hctof","neutron time of flight",1000,0,100);
////将数据填入刚刚声明的histogram中，Fill(x,y,z)	
	hctof->Fill(ctof);//该命令一般放入循环体中

////将声明的histogram写入root文件；
	hctof->Write();
////将tree中Branch有关数据写入root文件；
	opt->Write();
////关闭root文件
	opf->Close()；
```

## 读取ROOT的tree数据，逐事件分析流程
**！！！注意输入tree与输出tree的差异，特别是Makefile编译方法时！！！**
1. 主函数；
2. 打开root文件，得到tree的TTree指针；
3. 声明tree结构中Branch所需的变量；
4. 将tree中Branch的数据的指针指向声明的变量；
5. 将数据写入新的ROOT文件（参照《创建带有tree结构的root文件》）；
6. 得到tree的事件总数，由循环体遍历每个事件；
7. 将第jentry个事件数据填入Branch中指向的变量，填入对应的fistogram与tree；
7. 画图/关闭tree，写入新的tree，关闭新的root，关闭老的root；

详细代码见李老师数据分析课homework1.1中的[homework1.1-sum](https://dragon-xi.github.io/zhli-data-ana-homework/homework1.1/homework1.1-sum.html)

**代码示例**

```c++
////打开名为tree.root的文件（只读），并指定指针为ipf，脚本最后注意关闭文件ipf->Close();
	TFile * ipf = new TFile("tree.root"，"read");
////检验tree.root文件是否存在；
	if (ipf->IsZombie())
	{
		cout << "Error opening file" << endl;
		exit(-1);
	}
////为了避免多个root数据扰乱，加入此代码;
	ipf->cd();
////得到名字为tree的TTree指针，tree的名字可由.ls得到
TTree * tree = (TTree*)ipf->Get("tree");

////声明tree的Branch变量（可由tree->Print()得到或通过MakeClass命令生成相关文件）
	Double_t x;
	Int_t pid;
////将变量指向tree中Branch的地址(注意此处与创建Branch格式不同，没有数据类型）
	tree->SetBranchAddress("x",&x);
	tree->SetBranchAddress("pid",&pid);

////---将新数据写入新的ROOT文件（参考创建带有tree结构的root文件大致流程）
////---新变量声明；
	Double_t x_cali;
////---创建新root、新tree；
	TFile * opf = new TFile("tree.root","recreate");
	TTree * opt = new TTree("tree","tree structure");
////---新变量地址指向新的tree的Branch；
	opt->Branch("x_cali",&x_cali,"x_cali/D");

////逐事件读取tree的branch数据
////得到tree的事件总数，利用for循环遍历事件
	//if (fChain == 0) return;
	Long64_t nentries = tree->GetEntries();//！！！此处tree为输入文件的tree！！！
	Long64_t nbytes = 0, nb = 0;
	for(Long64_t jentry=0;jentry<nentries;jentry++)
	 {
////循环体中提取第n个事件的信息(在此之前已经声明了该root文件中的tree的Branch与变量地址的对应关系)；
		tree->GetEntry(jentry);//！！！此处tree为输入文件的tree！！！
		Long64_t ientry = LoadTree(jentry);
		if (ientry < 0) break;
		nb = tree->GetEntry(jentry);	nbytes += nb;
////---新的变量 = 旧数据处理后结果（参考创建带有tree结构的root文件大致流程）
		x_cali=f(x);
////---将新的结果填入tree中；（参考创建带有tree结构的root文件大致流程）
		opt->Fill();////！！！此处tree为输出文件的tree！！！

////为了检查数据处理进程，可添加
		if(jentry%100000==0) cout<<"process"<<jentry<<"of"<<nentries<<endl;
	}

////关闭读入的root文件
	ipf->Close();
////---将tree中Branch有关数据写入root文件；
	opt->Write();
////---关闭root文件
	opf->Close()；
```

## 头文件
**自己编写可编译执行文件时**
- <>:为ROOT、C++库函数，
- ""：为自己编写的.h,.c等函数
- C++(相关函数头文件请查看：http://www.cplusplus.com/)

```c++
#include <iostream>	//涉及cout，cin
#include <sstream>	//sprintf、string
using namespace std;	//std标准输入输出

//ROOT(相关函数头文件请查看：https://root.cern/doc/master/index.html)
#include <TROOT.h>	//ROOT标准
#include <TChain.h>	//Chain：多个root首位连结（数据结构要求一样）
#include <TFile.h>	//TFile：打开、创建.root文件
#include <TTree.h>	//TTree *tree:创建tree结构；
#include <TH2.h>	//TH2I,TH2D,TH2F:创建二维histogram；
#include <TStyle.h>	//gStyle->SetPalette()
#include <TCanvas.h>	//TCanvas * c1 = new TCanvas(""):创建画布；
#include <TF1.h>	//pol1、gaus，TF1 * f1:调用函数以及创建函数；
#include <TFitResult.h>	//拟合、调整参数、提取拟合：参数
#include <TString.h>	//sstream的root化；
#include <TGraph.h>	//TGraph * g1:创建graph，注意区graph vs histogram；
```

## C++/ROOT常用字符输入输出

```c++
    //c++
    sprintf(charname,"c1data%04d.root",i);
	string path = "/home/wangdongxi/ana_xi/caliprepare/c2forcaliroot/";

    //root
    //声明
    TString name;
    //赋值
    name = "内容"；
    nema.Form = ("内容1%s%d内容2", 变量1, 变量2)；  %s/%d为变量格式，具体可见格式化输出；
    //调用
    nema.Data()
```

## MakeClass方法
**作用**
- 自动生成该root文件的数据结构，并生成循环执行脚本（Loop()函数）；
- 直接添加想要的内容，并在Loop()中添加分析程序，无需从头编写（如2.2节）；
- 对于编写可编译执行程序，采用继承的方法链接该文件，免去了键入输入文件tree结构的内容；

详细方法见李老师数据处理课homework2.1中的[exercise2.4](https://dragon-xi.github.io/zhli-data-ana-homework/homework2.1/example_li/exercise2.4.html)以及[作业分析代码](https://dragon-xi.github.io/zhli-data-ana-homework/homework2.1/homework2.1_Code.html)

### 调用MakeClass方法

```c++
	tree->MakeClass("ppac");//输出ppac.h和ppac.C文件
	 修改ppac.C中Loop部分后
	gROOT->ProcessLine(".L ppac.C");//ROOT命令行.L ppac.C
	ppac t;//实体化
	t.GetEntry(13);//运行ppac类中GetEntry()函数
	t.Show();//运行ppac类中Show()函数
	t.Loop();//运行ppac类中Loop()函数,逐个事件分析数据；
```

### MakeClass with TChai

```c++
	TChain* chain=new TChain("tree");//chain即位单个root文件中的tree
	chain->Add("run0001.root");
	chain->Add("run0002.root");
	chain->Add("run0003.root");
	chain->Add("run0004.root");
	chain->Add("run0005.root");
	chain->MakeClass("ppac_chain");//输出chain脚本文件
```


## 探测器刻度（扣本地、寻峰、排序对应、拟合、提取参数）

具体代码请看李老师数据处理课homework3.1中的[拟合与刻度](https://dragon-xi.github.io/zhli-data-ana-homework/homework3.1/homework3.1_2_cali.html)

1. 扣本底--TSpectrum (选择使用)
```c++
////用法：Int_t TSpectrum::Background(const TH1 * hin, 	//input 1-d histogram
////					Int_t niter = 2,		//光滑度
////					Option_t * option = “”)	//功能项

	 TSpectrum *s=new TSpectrum(500);				//实例化，500：maximum number of peaks
	 TH1F *hb=(TH1F*)s->Background(h0,30,"same");		//拟合h0本底，并传递给hb，30-本底光滑程度, 用户进行调节
	 h0->Clone("hh");		//克隆h0，命名为hh
	 hh->Add(hh,hb,1,-1);		//将hh与hb叠加，TH1F * h0 = Add(TH1F * h1, TH1F * h2, c1, c2)
					//即h0 = h1 * c1 + h2 * c2
```		

2. 寻峰--TSpectrum
```c++
////用法：Int_t TSpectrum::Search(const TH1 * hin, 		//input 1-d histogram
////					Double_t sigma = 2,		//光滑度
////					Option_t * option = “”,		//功能项
////					Double_t threshold = 0.05)	//count>ther * Max of Counts of peak
	Int_t nfound=s->Search(hh,2,"",0.05); 	//寻峰的结果（个数）传递给nfound；
	Double_t *xpeaks,*ypeaks;			//指针
	xpeaks=s->GetPositionX();			//寻峰的结果（Mean）传递给xpeak；
	ypeaks=s->GetPositionY();			//寻峰的结果（Counts）传递给ypeak；
	//A pointer to the polymarker object can be retrieved later via:
	TList *functions = hin->GetListOfFunctions();
	TPolyMarker *pm = (TPolyMarker*)functions->FindObject("TPolyMarker");
```

3. 峰位排序-map
```c++
//	***map是STL（中文标准模板库）的一个关联容器。
//	***map提供一对一的数据处理，key-value键值对，其类型可以自己定义，第一个称为关键字，第二个为关键字的值
//	***multimap可以允许多个key值是相同的，而map的key值不允许重复。
//	***map,multimap, 按照key值的大小自动排序，key值有小到大。
	
	//定义map变量
	map<Int_t,Double_t> m1;//第一个是键（key）的类型，第二个是值（value）的类型
	multimap<Int_t,Double_t>mul1;
	//赋值
	TRandom3 *g=new TRandom3(0);
	for(int i=0;i<nfound;i++)
	{
		int key=xpeaks[i];
		double value=ypeaks[i];
		m1.insert(make_pair(key,value));//插入key，value值
		mul1.insert(make_pair(key,value));
		mul1.insert(make_pair(key,value));//重复插入key，value值		
	}
	//查看map中有多少数据
	cout<<m1.size()<<","<<mul1.size();
	//查找元素--通过迭代器im1进行访问
	//	***im1->first来访问key，使用im->second访问value
	//	***map.find(key)： 查找一个元素，返回键是key的映射的迭代器
	Int_t i = 762;
	auto im1=m1.find(i);
	if(im1!=m1.end()) 
		cout<<im1->first<<", "<<im1->second<<endl;
	else 
		cout<<"key="<<i<<" is not found."<<endl;
	//正向遍历
	//	***m1.begin()返回指向map头部的迭代器
	//	***m1.end()返回指向map末尾的迭代器
	// Hearder--->tail
	int num=0;
	cout<<"map elements:"<<endl;
	for(auto im=m1.begin();im!=m1.end();im++)
	{
		if(num>20) break;
		cout<<"	"<<im->first<<", "<<im->second<<endl;
		num++;
	}

	//反向遍历
	//	***m1.rbegin()返回指向map尾部的逆向迭代器
	//	***m1.rend()返回指向map头部的逆向迭代器
	// tail --->	Hearder
	num=0;
	cout<<"map elements:"<<endl;
	for(auto im=m1.rbegin();im!=m1.rend();im++)
	 {
		if(num>20) break;
		cout<<"	"<<im->first<<", "<<im->second<<endl;
		num++;
	}
	//其他成员函数
	//	***maps.clear()清空
	//	***maps.erase()删除一个元素
	//	***maps.lower_bound() 会返回一个迭代器，它指向键值和参数相等或大于参数的第一个元素。
	//	***maps.upper_bound() 也返回一个迭代器，它指向键值大于函数参数的第一个元素。
```

## 在root文件中创建/读取子文件夹

### 创建
1. 用TDirectoryFIle创建目录结构；
	
	- 在文件根目录下创建dir1目录：
        ```c++
        fout->cd();//fout为root文件指针，非tree指针哦！！！
		TDirectoryFile * dir1 = new TDirectoryFile("dir1","dir1");
        ```
	- 在dir1目录下创建子目录：
        ```c++
		dir1->cd();//dir1为目录指针；
		TDirectoryFile * dir1sub = new TDirectoryFile("dir1sub","dir1sub");
        ```
2. 创建目录完成后，分别在各个目录下（包括直接在fout文件中）声明要存储的对象并分配内存空间（new）；
	
	- 可以在目录结构中存储TH，TGraph，TTree等：
        ```c++
		TH1I * h0, * h1, * h2, * h1sub;
		fout->cd(); h0 = new TH1I();
		dir1->cd(); h1 = new TH1I();
		dir2->cd(); h2 = new TH1I();
		dir1sub->cd(); h1sub= new TH1I();
        ```
	- 填图：
        ```c++
		h0->Fill();
		...
        ```
	- 写入root文件：
        ```c++
		fout->Write();
        ```
        
        
### 读取

1. 打开带有目录的root文件：
```c++
	TFile * fin = new TFile("test1.root");
```
2. 查看root文件内容：
```c++
	fin->ls();
```
3. 得到目录中的TH图的指针：
```c++
	TH1I * h0a = (TH1I*)fin->Get("h0");//得到root内的TH
	TH1I * h1a = (TH1I*)fin->Get("dir1/h1");//得到root内dir1目录下的TH
	TH1I * h2a = (TH1I*)fin->Get("dir2/h2");//得到root内dir2目录下的TH
	TH1I * h1suba = (TH1I*)fin->Get("dir1/dir1sub/h1sub");//得到root内dir1的子目录dir1sub下的TH
```
4. 画图：
```c++
	h1suba->Draw();
	c1->Draw();
```

# 画图操作命令（画图、拟合、坐标修改、颜色更换等等）

## 脚本画图

```c++
////创建指针为c1的画布，c1可根据需要命名；
	TCanvas *c1 = new TCanvas();
////清空画布c1；	
	c1->Clear();
////画tree中histogram，
////	第一个双引号：内容+座标轴范围与bin，alias为该条件别名；
////	第二个双引号：画图的条件、符合、CUT等；
////	第三个双引号：option，如colz等；			
	tree->Draw("td-tu>>htx(500,-20,50)","","");
////执行画图
////脚本模式中每次更改设置修改画图内容，都需要执行此命令更新执行画图；	
////***后面只说功能，不再写此命令***
	c1->Draw();
```

## 得到histogram指针，以便对图像修改、拟合等操作（两种方法注意区分）

- 得到ipf文件(打开的root文件指针)内hctof数据指针的方法；root环境下可直接用hctof->Draw()；
```c++
	TH1D * hh = (TH1D *)ipf->Get("hctof");
	hh->Draw();//显示hh文件内的histogram
```

- 得到tree->Draw()中histogram别名的指针，通过指针进行进一步操作	
```c++
	tree->Draw("td-tu>>htx(500,-20,50)","","");
	TH1D * htx = (TH1D *)gROOT->FindObject("htx");
	htx->Draw();
```
- 将TH图转换成TGraph，并得到TGraph中事件数
```c++	
    TGraph *gtarget = new TGraph(tree->GetSelectedRows(),tree->GetV2(), tree->GetV1());
	Double_t event_all = gtarget->GetN();
```

## 叠图

- 方法一：坐标信息由same前的Draw()定死了；
```c++
    hty->Draw();
    htx->Draw("same");
```

- 方法二：根据每个图的大小，自动调节坐标范围；
```c++
    THStack *hs = new THStack("hs","test stacked histograms");
	hs->Add(htu);
	hs->Add(htd);
	hs->Draw("nostack");//无此命令会自动归一到第一个图的坐标；
	c1->Draw();
```

## 填加图例

```c++
	auto legend = new TLegend(0.7, 0.7, .9, .9);
		 legend->SetHeader("Qu with different PID","C"); // option "C" allows to center the header
		 legend->AddEntry(hquall,"Qu all","l");
		 legend->AddEntry(hqu0,"Q_u for gamma","l");
		 legend->AddEntry(hqu1,"Q_u for neutron","l");
		 legend->AddEntry(hqu2,"Q_u for proton","l");
		 legend->AddEntry(hqu3,"Q_u for pedal","l");
		 legend->Draw();
```

## histogram拟合与结果提取

```c++
////tree->Draw()中可在bin前加入别名，后续用此指针(htx)画图、拟合、same等；
	tree->Draw("td-tu>>htx(500,-20,50)","","");
////创建拟合函数，也可用root自带的gaus、pol等；
	TF1 *f1 = new TF1("f1","[0]*TMath::Exp(-0.5*((x-[1])/[2])^2)",39.5,43);
////初始化拟合参量
	f1->SetParameter(0,-350);//大致估计范围
	f1->SetParameter(1,41.5);
	f1->SetParameter(2,0.5);
////拟合histgram,Fit("function","option","",range_low,range_high)
	htx->Fit("f1","R");
	htx->Fit("gaus","","",0，200)；//高斯
	htx->Fit("pol1","","",0，200)；//多项式pol1-9；
////提取拟合结果方法
	TF1 *fgaus[2];
	Double_t ped[2],sigma[2];//u,d
	TString sq[2]={"qu","qd"};
	fgaus[0]=hquall->GetFunction("gaus");//得到拟合函数的指针
	fgaus[1]=hqdall->GetFunction("gaus");
	for(int i=0;i<2;i++) 
	{
		ped[i]=fgaus[i]->GetParameter(1);//得到拟合函数的第二个参数
		sigma[i]=fgaus[i]->GetParameter(2);
////TString的格式化输出。用法与printf一致。
		TString ss;
		ss.Form("ped_%s=%.2f, sigma_%s=%.2f",sq[i].Data(),ped[i],sq[i].Data(),sigma[i]); 
		cout<<ss<<endl;
	 }
```

## 其他功能


- histogram不显示传递误差：histogram name->Sumw2(0);
```c++
dtd->Sumw2(0)；
```

- 更改histgram线颜色
```c++
	htx->SetLineColor(kGreen);
	htx->SetLineColor(kRed);
	htx->SetLineColor(kBlue);
	htx->SetLineColor(kGray);
	htx->SetLineColor(kBlack);
```

- 将htx的二维图向X轴投影
```c++
TH1D * htx1 = htx->ProjectionX("projx of htx");
```

- 清空c1画布的内容；
```c++
c1->Clear();
```

- 将坐标轴改为log坐标；
```c++
c1->SetLogx();	gPad->SetLogx();
c1->SetLogy();	gPad->SetLogy();
c1->SetLogz();	gPad->SetLogz();
```

- 将坐标轴改为正常坐标；
```c++
    c1->SetLogx(0);	gPad->SetLogx(0);
	c1->SetLogy(0);	gPad->SetLogy(0);
	c1->SetLogz(0);	gPad->SetLogz(0);
```

- 更换sea color颜色；
```c++
gStyle->SetPalette(1);	
```

- 设置参数的别名：tree->SetAlias()
```c++
    TString squa,sqda;
	squa.Form("iqu-%f",ped[0]);
	sqda.Form("iqd-%f",ped[1]);
	tree->SetAlias("qua",squa.Data());
	tree->SetAlias("qda",sqda.Data());
	TString stcut="itu>0&&itu<4000&&itd>0&&itd<4000";

	scut=scut+" && "+stcut;
	tree->Draw("itd-itu:log(qua/qda)",scut.Data(),"colz");
	c1->SetLogz();
	c1->SetLogy(0);
	c1->Draw();
```

- 在TH（TH1D * tdiff）中提取bin相关信息
```c++
    tdiff->GetNbinxX();//提取TH中bin的个数
	tdiff->GetBinContent([num]);//提取TH中num处bin的计数；
	tdiff->GetBinLowEdge([num]);//提取TH中num处bin的低边界；
```

- 得到图中计数
```c++
tree->GetEntries("GCUT");//得到给定条件下，上一tree->Draw()命令得到的图中的事件数；
```

## 颜色以及功能项

### [Colors](https://root.cern.ch/doc/master/classTAttLine.html)
![avatar](./root_color.png)

### [width and style of Line](https://root.cern.ch/doc/master/classTAttLine.html)
![avatar](./root_line.png)

### [width and style of Marker](https://root.cern.ch/doc/master/classTAttMarker.html)
![avatar](./root_marker.png)

### [option of tree draw and graph](https://root.cern.ch/doc/master/classTHistPainter.html)
![avatar](./root_tree_draw_option.png)
![avatar](./root_graph_option.png)

In [1]:
!jupyter nbconvert roottips_xi.ipynb --to html

[NbConvertApp] Converting notebook roottips_xi.ipynb to html


